In [77]:
import requests
import json
import pandas as pd
import time

In [78]:
# pip install xmltodict

In [79]:
import xmltodict

In [80]:
data_df = pd.DataFrame()

In [81]:
url = 'http://safemap.go.kr/openApiService/data/getFrfireSttusData.do'

In [82]:
for i in range(1, 8) : 
    params ={'serviceKey' : '개인 api키 사용', 
             'pageNo' : i, 
             'numOfRows' : '1000'}

    response = requests.get(url, params=params)
    data = response.content.decode('utf-8')
    temp_dict = xmltodict.parse(data)
    temp_df = pd.DataFrame(temp_dict['response']['body']['items']['item'])
    data_df = pd.concat([data_df, temp_df])
    time.sleep(1)

In [83]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6397 entries, 0 to 396
Data columns (total 21 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   OBJT_ID    6397 non-null   object
 1   OCCU_YEAR  6397 non-null   object
 2   OCCU_MT    6397 non-null   object
 3   OCCU_DE    6397 non-null   object
 4   OCCU_TM    6397 non-null   object
 5   OCCU_DAY   6397 non-null   object
 6   OCCU_DATE  6397 non-null   object
 7   END_YEAR   6397 non-null   object
 8   END_MT     6397 non-null   object
 9   END_DE     6397 non-null   object
 10  END_TM     6397 non-null   object
 11  ADRES      6397 non-null   object
 12  RN_ADRES   6397 non-null   object
 13  RESN       6397 non-null   object
 14  AR         6397 non-null   object
 15  AMOUNT     6397 non-null   object
 16  CTPRVN_CD  6397 non-null   object
 17  SGG_CD     6397 non-null   object
 18  EMD_CD     6397 non-null   object
 19  X          6397 non-null   object
 20  Y          6397 non-null   object
dt

In [84]:
data_df = data_df.reset_index(drop=True)

In [85]:
data_df.columns = ['일련번호', '발생년도', '발생월', '발생일', '발생시간', '발생요일',\
                   '발생년월일', '종료년도', '종료월', '종료일', '종료시간', '지번주소',\
                   '도로명주소', '사유', '피해면적', '피해금액', '시도코드', '시군구코드',\
                   '읍면동코드', 'X좌표', 'Y좌표']

In [86]:
# data_df.to_csv('산불위치rawdata.csv', index=False, encoding='utf-8')
# data_df = pd.read_csv('산불위치rawdata.csv', encoding='utf-8')

In [87]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6397 entries, 0 to 6396
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   일련번호    6397 non-null   object
 1   발생년도    6397 non-null   object
 2   발생월     6397 non-null   object
 3   발생일     6397 non-null   object
 4   발생시간    6397 non-null   object
 5   발생요일    6397 non-null   object
 6   발생년월일   6397 non-null   object
 7   종료년도    6397 non-null   object
 8   종료월     6397 non-null   object
 9   종료일     6397 non-null   object
 10  종료시간    6397 non-null   object
 11  지번주소    6397 non-null   object
 12  도로명주소   6397 non-null   object
 13  사유      6397 non-null   object
 14  피해면적    6397 non-null   object
 15  피해금액    6397 non-null   object
 16  시도코드    6397 non-null   object
 17  시군구코드   6397 non-null   object
 18  읍면동코드   6397 non-null   object
 19  X좌표     6397 non-null   object
 20  Y좌표     6397 non-null   object
dtypes: object(21)
memory usage: 1.0+ MB


In [88]:
data_df.isna().sum()

일련번호     0
발생년도     0
발생월      0
발생일      0
발생시간     0
발생요일     0
발생년월일    0
종료년도     0
종료월      0
종료일      0
종료시간     0
지번주소     0
도로명주소    0
사유       0
피해면적     0
피해금액     0
시도코드     0
시군구코드    0
읍면동코드    0
X좌표      0
Y좌표      0
dtype: int64

In [89]:
from pyproj import Transformer

# EPSG:3857 -> EPSG:4326 (위도, 경도)로 변환
transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326")

lat_list = []
lon_list = []

for x, y in zip(data_df['X좌표'], data_df['Y좌표']) :
    lat, lon = transformer.transform(x, y)
    lat_list.append(lat)
    lon_list.append(lon)

In [90]:
data_df['위도'] = lat_list
data_df['경도'] = lon_list

In [91]:
data_df[['지번주소', 'X좌표', 'Y좌표', '위도', '경도']].head()

,지번주소,X좌표,Y좌표,위도,경도
0,경상남도 함안군 칠북면 봉촌리 산270-1,14309755.68,4213756.081,35.366184,128.546722
1,경상남도 거제시 일운면 소동리 산32-2,14327001.75,4142330.165,34.841262,128.701646
2,경상남도 거제시 일운면 소동리 산75-6,14324658.43,4141242.186,34.833240,128.680596
3,경상남도 산청군 단성면 입석리 산20,14239111.63,4204701.45,35.299827,127.912116
4,경상남도 산청군 생비량면 도리 산164,14254878.82,4208490.867,35.327604,128.053755


In [92]:
data_df = data_df.drop(columns=['일련번호', '발생년월일', '시도코드', '시군구코드', '읍면동코드',\
                                'X좌표', 'Y좌표', '도로명주소'])

In [93]:
data_df.head()

,발생년도,발생월,발생일,발생시간,발생요일,종료년도,종료월,종료일,종료시간,지번주소,사유,피해면적,피해금액,위도,경도
0,2011,01,01,1340,토요일,2011,01,01,1630,경상남도 함안군 칠북면 봉촌리 산270-1,입산자실화,1,52000,35.366184,128.546722
1,2011,01,01,1437,토요일,2011,01,01,1525,경상남도 거제시 일운면 소동리 산32-2,입산자실화,0.05,4096,34.841262,128.701646
2,2011,01,01,1607,토요일,2011,01,01,1620,경상남도 거제시 일운면 소동리 산75-6,입산자실화,0.02,2291,34.833240,128.680596
3,2011,01,08,1227,토요일,2011,01,08,1255,경상남도 산청군 단성면 입석리 산20,쓰레기소각,0.01,3730,35.299827,127.912116
4,2011,01,10,2130,월요일,2011,01,11,0130,경상남도 산청군 생비량면 도리 산164,입산자실화,0.1,4740,35.327604,128.053755


In [94]:
def trans_date(years, months, days, hours) :
    date_list = []
    for y, m, d, h in zip(years, months, days, hours) : 
        format = y + '-' + m + '-' + d + ' ' + h[:2] + ':' + h[2:]
        date_list.append(pd.to_datetime(format))
    return date_list

In [95]:
data_df['발생시각'] = trans_date(data_df['발생년도'], data_df['발생월'],data_df['발생일'],data_df['발생시간'])

In [96]:
# data_df['종료시각'] = trans_date(data_df['종료년도'], data_df['종료월'],data_df['종료일'],data_df['종료시간'])
# DateParseError: day is out of range for month: 2014-02-00 46:00, at position 0 에러발생

In [97]:
data_df['종료일'].value_counts()

종료일
09    261
06    246
23    242
04    236
02    231
26    231
05    227
10    227
28    226
03    223
27    221
11    221
24    219
19    218
22    217
12    211
16    209
01    208
25    205
08    197
21    194
17    193
15    191
13    187
07    182
18    174
29    170
14    170
30    168
20    167
31    123
00      2
Name: count, dtype: int64

In [98]:
data_df[data_df['종료일']=='00']

,발생년도,발생월,발생일,발생시간,발생요일,종료년도,종료월,종료일,종료시간,지번주소,사유,피해면적,피해금액,위도,경도,발생시각
768,2014,02,04,0243,화요일,2014,02,00,4600,울산광역시 울주군 언양읍 대곡리 861-4,조사중,0.7,13485,35.609134,129.176236,2014-02-04 02:43:00
1089,2014,05,06,2143,화요일,2014,05,00,7200,인천광역시 부평구 부개동 산7-1,입산자실화 추정,0.1,0,37.480407,126.730434,2014-05-06 21:43:00


In [99]:
data_df[data_df['종료시간'].astype('int')>=2400]

,발생년도,발생월,발생일,발생시간,발생요일,종료년도,종료월,종료일,종료시간,지번주소,사유,피해면적,피해금액,위도,경도,발생시각
768,2014,02,04,0243,화요일,2014,02,00,4600,울산광역시 울주군 언양읍 대곡리 861-4,조사중,0.7,13485,35.609134,129.176236,2014-02-04 02:43:00
1089,2014,05,06,2143,화요일,2014,05,00,7200,인천광역시 부평구 부개동 산7-1,입산자실화 추정,0.1,0,37.480407,126.730434,2014-05-06 21:43:00


In [100]:
# 이상치 data drop
data_df = data_df.drop([768, 1089])
data_df = data_df.reset_index(drop=True)

In [101]:
data_df['종료시각'] = trans_date(data_df['종료년도'], data_df['종료월'],data_df['종료일'],data_df['종료시간'])

In [102]:
import math

data_df['진화시간'] = data_df['종료시각']-data_df['발생시각']
data_df['진화시간'] = data_df['진화시간'].apply(lambda x : x.total_seconds() / 60)

In [ ]:
data_df = data_df.drop(data_df[data_df['진화시간']<0].index)
data_df.reset_index(drop=True)

In [108]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6358 entries, 0 to 6394
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   발생년도    6358 non-null   object        
 1   발생월     6358 non-null   object        
 2   발생일     6358 non-null   object        
 3   발생시간    6358 non-null   object        
 4   발생요일    6358 non-null   object        
 5   종료년도    6358 non-null   object        
 6   종료월     6358 non-null   object        
 7   종료일     6358 non-null   object        
 8   종료시간    6358 non-null   object        
 9   지번주소    6358 non-null   object        
 10  사유      6358 non-null   object        
 11  피해면적    6358 non-null   object        
 12  피해금액    6358 non-null   object        
 13  위도      6358 non-null   float64       
 14  경도      6358 non-null   float64       
 15  발생시각    6358 non-null   datetime64[ns]
 16  종료시각    6358 non-null   datetime64[ns]
 17  진화시간    6358 non-null   float64       
dtypes: datetime64

In [110]:
data_df = data_df.drop(columns=['발생년도', '발생월', '발생일', '발생시간', '발생요일',\
                                '종료년도', '종료월', '종료일', '종료시간', '종료시각'])

In [111]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6358 entries, 0 to 6394
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   지번주소    6358 non-null   object        
 1   사유      6358 non-null   object        
 2   피해면적    6358 non-null   object        
 3   피해금액    6358 non-null   object        
 4   위도      6358 non-null   float64       
 5   경도      6358 non-null   float64       
 6   발생시각    6358 non-null   datetime64[ns]
 7   진화시간    6358 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 447.0+ KB


In [112]:
data_df['진화시간'] = data_df['진화시간'].astype('int')

In [113]:
data_df['피해면적'] = data_df['피해면적'].astype('float')

In [114]:
data_df['피해금액'] = data_df['피해금액'].astype('float')

In [115]:
pd.set_option('display.max_rows', None)

In [116]:
data_df['사유'].value_counts()

사유
입산자실화                                        651
원인미상                                         608
입산자실화추정                                      395
쓰레기소각                                        357
입산자 실화                                       284
입산자 실화추정                                     248
담뱃불실화                                        229
농산폐기물소각                                      220
쓰레기 소각                                       200
등산객실화                                        121
기타(직접입력)                                     113
농산폐기물 소각                                     103
입산자실화 추정                                      97
조사중                                           91
논밭두렁소각                                        87
성묘객실화                                         80
밭두렁소각                                         60
담뱃불실화추정                                       49
주택화재비화                                        48
밭두렁 소각                                        43
낙뢰               

In [117]:
data_df['사유'] = \
    data_df['사유'].apply(lambda x : '담뱃불 실화' if ( ('담배' in x) | ('담뱃' in x) ) else x)

In [118]:
# data_df[data_df['사유'].str.contains('담배|담뱃', regex=True]['사유'].value_counts()

In [119]:
# data_df[data_df['사유'].str.contains('소각|폐기|두렁|쓰레기', regex=True)]['사유'].value_counts()

In [120]:
data_df['사유'] = \
    data_df['사유'].apply(lambda x : '논밭/쓰레기 소각' if ( ('소각' in x) | ('폐기' in x) | ('두렁' in x) |\
                                                  ('쓰레기' in x) ) else x)

In [121]:
# data_df[data_df['사유'].str.contains('화재|주택|건축|시설|공사')]['사유'].value_counts()

In [122]:
data_df['사유'] = \
    data_df['사유'].apply(lambda x : '건축/시설물 화재' if ( ('화재' in x) | ('주택' in x) | ('건축' in x) |\
                                                ('시설' in x) | ('공사' in x)) else x)

In [123]:
# data_df[data_df['사유'].str.contains('재.*[투기|처리|취급]|불씨|화목|보일러|용접|연탄|아궁이|숯',\
# regex=True)]['사유'].value_counts()

In [124]:
data_df['사유'] = \
    data_df['사유'].apply(lambda x : '재/불씨/용접 등' \
                        if ( ('재' in x) & ( ('투기' in x) | ('처리' in x) | ('취급' in x) ) |\
                            ('불씨' in x) | ('화목' in x) | ('보일러' in x) | ('용접' in x) | \
                           ('연탄' in x) | ('아궁이' in x) | ('숯' in x)) else x)

In [125]:
# data_df[data_df['사유'].str.contains('전기|전선|누전|스파크', regex=True)]['사유'].value_counts()

In [126]:
data_df['사유'] = \
    data_df['사유'].apply(lambda x : '누전/스파크' \
                        if ( ('전기' in x) | ('전선' in x) | ('누전' in x) | ('스파크' in x) ) else x)

In [127]:
data_df['사유'] = data_df['사유'].apply(lambda x : '낙뢰' if ('낙뢰' in x) else x)

In [128]:
data_df['사유'] = data_df['사유'].apply(lambda x : '불장난' if ('장난' in x) else x)

In [129]:
data_df['사유'] = data_df['사유'].apply(lambda x : '방화' if ('방화' in x) else x)

In [130]:
data_df['사유'] = data_df['사유'].apply(lambda x : '원인미상' if ( ('미상' in x) | ('조사' in x) ) else x)

In [131]:
# data_df[data_df['사유'].str.contains('입산|등산|성묘|무속|채취|낚시|취사', regex=True)]['사유'].value_counts()

In [132]:
data_df['사유'] = \
    data_df['사유'].apply(lambda x : '입산자 기타 실화' \
                        if ( ('입산' in x) | ('등산' in x) | ('성묘' in x) | ('취사' in x) |\
                           ('무속' in x) | ('채취' in x) | ('낚시' in x)) else x)

In [133]:
data_df[data_df['사유'].str.contains('미상|조사', regex=True)]['사유'].value_counts()

사유
원인미상    770
Name: count, dtype: int64

In [134]:
# data_df[data_df['사유'].str.contains('산업|사업|작업', regex=True)]['사유'].value_counts()

In [135]:
data_df['사유'] = data_df['사유'].apply(lambda x : '작업/산업현장' \
                                    if ( ('산업' in x) | ('사업' in x) | ('작업' in x) ) else x)

In [136]:
data_df['사유'].value_counts()

사유
입산자 기타 실화             2170
논밭/쓰레기 소각             1683
원인미상                   770
담뱃불 실화                 468
건축/시설물 화재              399
재/불씨/용접 등              294
기타(직접입력)               113
작업/산업현장                 92
누전/스파크                  81
방화                      77
낙뢰                      49
불장난                     26
풍등                       3
화기물 취급 부주의               2
기계톱과열                    2
분신자살 추정                  2
인근축사 발화 추정               1
묘지주변 정리 중 발화             1
군 발물처리                   1
새 ?기위해 터트린 폭죽            1
군사훈련전이                   1
변압기추락                    1
야영장모닥불                   1
군부대실화추정                  1
난로재 실화                   1
군사훈현                     1
농막에서 발화 추정               1
전신주도복에의한발화(추정)           1
담대꽁초투기                   1
묘지사초                     1
밧데리폭발                    1
양수기과열                    1
온수공급호스연결부주의              1
건출물실화(가구공장 전소)           1
농기계 수리중 폭발               1
군부대 조명탄(신호킷 추정)          1
캠핑객 실화                   

In [137]:
r_list = ['입산자 기타 실화', '논밭/쓰레기 소각', '원인미상', '담뱃불 실화', '건축/시설물 화재',\
          '재/불씨/용접 등', '작업/산업현장', '누전/스파크', '방화', '낙뢰', '불장난']

In [138]:
data_df['사유'] = data_df['사유'].apply(lambda x : '기타' if (x not in r_list) else x)

In [139]:
data_df['사유'].value_counts()

사유
입산자 기타 실화    2170
논밭/쓰레기 소각    1683
원인미상          770
담뱃불 실화        468
건축/시설물 화재     399
재/불씨/용접 등     294
기타            249
작업/산업현장        92
누전/스파크         81
방화             77
낙뢰             49
불장난            26
Name: count, dtype: int64

In [140]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6358 entries, 0 to 6394
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   지번주소    6358 non-null   object        
 1   사유      6358 non-null   object        
 2   피해면적    6358 non-null   float64       
 3   피해금액    6358 non-null   float64       
 4   위도      6358 non-null   float64       
 5   경도      6358 non-null   float64       
 6   발생시각    6358 non-null   datetime64[ns]
 7   진화시간    6358 non-null   int32         
dtypes: datetime64[ns](1), float64(4), int32(1), object(2)
memory usage: 422.2+ KB


In [141]:
data_df.to_csv('산불발생위치.csv', index=False, encoding='utf-8')